##### Copyright 2023 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Buffers de replay

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/5_replay_buffers_tutorial">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/5_replay_buffers_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/5_replay_buffers_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/agents/tutorials/5_replay_buffers_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Introdução

Os algoritmos de aprendizado por reforço usam buffers de replay para armazenar trajetórias de experiência ao executar uma política em um ambiente. Durante o treinamento, os buffers de replay são consultados sobre um subconjunto das trajetórias (seja um subconjunto sequencial ou uma amostra) para dar "replay" na experiência do agente.

Neste colab, exploramos dois tipos de buffers de replay: com suporte do python e do TensorFlow, que compartilham uma API. Nas seções a seguir, descrevemos a API, cada uma das implementações de buffer e como usá-las durante o treinamento da coleta de dados.


## Configuração

Instale o tf-agents caso não tenha feito isso.

In [ ]:
!pip install tf-agents


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np

from tf_agents import specs
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.networks import q_network
from tf_agents.replay_buffers import py_uniform_replay_buffer
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.specs import tensor_spec
from tf_agents.trajectories import time_step

## API Replay Buffer

A classe Replay Buffer tem a definição e os métodos a seguir:

```python
class ReplayBuffer(tf.Module):
  """Abstract base class for TF-Agents replay buffer."""

  def __init__(self, data_spec, capacity):
    """Initializes the replay buffer.

    Args:
      data_spec: A spec or a list/tuple/nest of specs describing
        a single item that can be stored in this buffer
      capacity: number of elements that the replay buffer can hold.
    """

  @property
  def data_spec(self):
    """Returns the spec for items in the replay buffer."""

  @property
  def capacity(self):
    """Returns the capacity of the replay buffer."""

  def add_batch(self, items):
    """Adds a batch of items to the replay buffer."""

  def get_next(self,
               sample_batch_size=None,
               num_steps=None,
               time_stacked=True):
    """Returns an item or batch of items from the buffer."""

  def as_dataset(self,
                 sample_batch_size=None,
                 num_steps=None,
                 num_parallel_calls=None):
    """Creates and returns a dataset that returns entries from the buffer."""


  def gather_all(self):
    """Returns all the items in buffer."""
    return self._gather_all()

  def clear(self):
    """Resets the contents of replay buffer"""

```

Observe que, quando o objeto do buffer de replay é inicializado, ele exige a `data_spec` dos elementos que armazenará. Essa especificação corresponde à `TensorSpec` dos elementos da trajetória que serão adicionados ao buffer. Essa especificação é geralmente adquirida ao observar a `agent.collect_data_spec` de um agente, que define os formatos, tipos e estruturas esperados por um agente durante o treinamento (falaremos mais disso depois).

## TFUniformReplayBuffer

`TFUniformReplayBuffer` é o buffer de replay geralmente usado no TF-Agents, portanto, será usado neste tutorial. No `TFUniformReplayBuffer`, o armazenamento do buffer de apoio é realizado pelas variáveis do TensorFlow, então faz parte do grafo de computação.

O buffer armazena lotes de elementos e tem uma capacidade máxima de `max_length` elementos por segmento de lote. Portanto, a capacidade de buffer total é `batch_size` x `max_length` elementos. Os elementos armazenados no buffer precisam ter a mesma especificação de dados. Quando o buffer de replay é usado para a coleta de dados, a especificação é a de coleta de dados do agente.


### Crie o buffer:

Para criar um `TFUniformReplayBuffer`, passamos o seguinte:

1. a especificação dos elementos de dados que serão armazenados pelo buffer
2. o `batch size` correspondente ao tamanho do lote do buffer
3. o número `max_length` de elementos por segmento de lote

Aqui está um exemplo de como criar um `TFUniformReplayBuffer` com especificações de dados de amostra, `batch_size` 32 e `max_length` 1000.

In [ ]:
data_spec =  (
        tf.TensorSpec([3], tf.float32, 'action'),
        (
            tf.TensorSpec([5], tf.float32, 'lidar'),
            tf.TensorSpec([3, 2], tf.float32, 'camera')
        )
)

batch_size = 32
max_length = 1000

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec,
    batch_size=batch_size,
    max_length=max_length)

### Escreva no buffer:

Para adicionar elementos ao buffer de replay, usamos o método `add_batch(items)`, onde `items` é uma lista/tupla/ninho de tensores que representa o lote de itens a serem adicionados ao buffer. Cada elemento de `items` precisa ter uma dimensão exterior igual a `batch_size`, e as dimensões restantes precisam obedecer às especificações de dados do item (igual às especificações de dados passadas ao construtor de buffer de replay).

Veja este exemplo de adição de um lote de itens


In [ ]:
action = tf.constant(1 * np.ones(
    data_spec[0].shape.as_list(), dtype=np.float32))
lidar = tf.constant(
    2 * np.ones(data_spec[1][0].shape.as_list(), dtype=np.float32))
camera = tf.constant(
    3 * np.ones(data_spec[1][1].shape.as_list(), dtype=np.float32))
  
values = (action, (lidar, camera))
values_batched = tf.nest.map_structure(lambda t: tf.stack([t] * batch_size),
                                       values)
  
replay_buffer.add_batch(values_batched)

### Leia o buffer:

Há três maneiras de ler dados do `TFUniformReplayBuffer`:

1. `get_next()` - retorna uma amostra do buffer. O tamanho do lote de amostra e o número de timesteps retornados podem ser especificados por argumentos para esse método.
2. `as_dataset()` - retorna o buffer de replay como um `tf.data.Dataset`. É possível criar um iterador de datasets e iterar as amostras dos itens no buffer.
3. `gather_all()` - retorna todos os itens no buffer como um Tensor de formato `[batch, time, data_spec]`

Confira abaixo exemplos de como ler o buffer de replay usando cada um desses métodos:

In [ ]:
# add more items to the buffer before reading
for _ in range(5):
  replay_buffer.add_batch(values_batched)

# Get one sample from the replay buffer with batch size 10 and 1 timestep:

sample = replay_buffer.get_next(sample_batch_size=10, num_steps=1)

# Convert the replay buffer to a tf.data.Dataset and iterate through it
dataset = replay_buffer.as_dataset(
    sample_batch_size=4,
    num_steps=2)

iterator = iter(dataset)
print("Iterator trajectories:")
trajectories = []
for _ in range(3):
  t, _ = next(iterator)
  trajectories.append(t)
  
print(tf.nest.map_structure(lambda t: t.shape, trajectories))

# Read all elements in the replay buffer:
trajectories = replay_buffer.gather_all()

print("Trajectories from gather all:")
print(tf.nest.map_structure(lambda t: t.shape, trajectories))


## PyUniformReplayBuffer

O `PyUniformReplayBuffer` tem a mesma funcionalidade que `TFUniformReplayBuffer`, mas, em vez de variáveis tf, os dados são armazenados em arrays do numpy. Esse buffer pode ser usado para a coleta de dados fora do grafo. O armazenamento de apoio no numpy pode facilitar para alguns aplicativos manipularem dados (como a indexação para a atualização de prioridades) sem usar variáveis do TensorFlow. No entanto, essa implementação não terá o benefício das otimizações de grafo com o TensorFlow.

Confira abaixo um exemplo de como instanciar um `PyUniformReplayBuffer` a partir das especificações de trajetória da política do agente.

In [ ]:
replay_buffer_capacity = 1000*32 # same capacity as the TFUniformReplayBuffer

py_replay_buffer = py_uniform_replay_buffer.PyUniformReplayBuffer(
    capacity=replay_buffer_capacity,
    data_spec=tensor_spec.to_nest_array_spec(data_spec))

## Uso de buffers de replay durante o treinamento

Agora que sabemos como criar, escrever itens e ler um buffer de replay, podemos usá-lo para armazenar trajetórias durante o treinamento dos nossos agentes.

### Coleta de dados

Primeiro, vamos ver como usar o buffer de replay durante a coleta de dados.

No TF-Agents, usamos um `Driver` (confira o tutorial Driver para mais detalhes) para coletar a experiência em um ambiente. Para usar um `Driver`, especificamos um `Observer`, que é uma função para o `Driver` executar quando ele recebe uma trajetória.

Portanto, para adicionar elementos de trajetória ao buffer de replay, adicionamos um observador que chama `add_batch(items)` para adicionar um lote de itens no buffer de replay.

Veja abaixo um exemplo disso com o `TFUniformReplayBuffer`. Primeiro, criamos um ambiente, uma rede e um agente. Em seguida, criamos um `TFUniformReplayBuffer`. Observe que as especificações dos elementos de trajetória no buffer de replay são iguais às especificações de coleta de dados do agente. Depois, configuramos o método `add_batch` como o observer do driver que coletará os dados durante nosso treinamento:


In [ ]:
env = suite_gym.load('CartPole-v0')
tf_env = tf_py_environment.TFPyEnvironment(env)

q_net = q_network.QNetwork(
    tf_env.time_step_spec().observation,
    tf_env.action_spec(),
    fc_layer_params=(100,))

agent = dqn_agent.DqnAgent(
    tf_env.time_step_spec(),
    tf_env.action_spec(),
    q_network=q_net,
    optimizer=tf.compat.v1.train.AdamOptimizer(0.001))

replay_buffer_capacity = 1000

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    agent.collect_data_spec,
    batch_size=tf_env.batch_size,
    max_length=replay_buffer_capacity)

# Add an observer that adds to the replay buffer:
replay_observer = [replay_buffer.add_batch]

collect_steps_per_iteration = 10
collect_op = dynamic_step_driver.DynamicStepDriver(
  tf_env,
  agent.collect_policy,
  observers=replay_observer,
  num_steps=collect_steps_per_iteration).run()

### Leitura de dados para um passo de treinamento

Após adicionar elementos de trajetória ao buffer de replay, podemos ler lotes de trajetórias a partir do buffer de replay para usar como dados de entrada para um passo de treinamento.

Aqui está um exemplo de como treinar com as trajetórias do buffer de replay em um loop de treinamento: 

In [ ]:
# Read the replay buffer as a Dataset,
# read batches of 4 elements, each with 2 timesteps:
dataset = replay_buffer.as_dataset(
    sample_batch_size=4,
    num_steps=2)

iterator = iter(dataset)

num_train_steps = 10

for _ in range(num_train_steps):
  trajectories, _ = next(iterator)
  loss = agent.train(experience=trajectories)
